In [ ]:
from pathlib import Path
import os
from febio_python.feb import FEBio_feb
import json

In [ ]:
feb_template_path = Path("./sample_files/sample_lv_template.feb")
json_data_path = Path("./sample_files/sample_lvhex_wfibers.json")

In [ ]:
with open(json_data_path, "r") as jfile:
  data = json.load(jfile)
data.keys()

In [ ]:
feb = FEBio_feb.from_file(feb_template_path)
feb

In [ ]:
feb.add_nodes([
  {"name": "LV", 
   "nodes": data["NODES"]
   },
  {"name": "AORTIC_RIM", 
   "nodes": data["BOUNDARY_CONDITIONS"]["AORTIC_BC"][1]["RIM_NODES"]
   },
  {"name": "MITRAL_RIM", 
   "nodes": data["BOUNDARY_CONDITIONS"]["MITRAL_BC"][1]["RIM_NODES"]
   }
  ])
feb.add_elements([
  {
    "name": "LV", 
    # "type": "tet4",
    "mat": "1",
    "elems": data["ELEMENTS"]["TETRA"]
    }
  ])
feb.add_nodesets(data["NODESETS"])
feb.add_surfaces(data["SURFACES"])
feb.add_discretesets(data["DISCRETE_SETS"])
feb.add_meshdata([
  {
    "elem_set": "LV", 
    "var": "mat_axis",
    "elems": {
      "a": data['MESH_ELEM_DATA']["LV_FIBERS"],
      "d": data['MESH_ELEM_DATA']["LV_SHEET"],
      }
    }
  ])

feb

In [ ]:
feb.replace_material_params({"1": {"a": 0.5}})

In [ ]:
feb.write("./sample_files/sample_lvhex_wfibers_mod.feb")